# Processing Data Download

This Jupyter Notebook serves the purpose to transform the downloaded data set (a collection of `.jpg` pictures) into interpretable input for the Machine Learning algorithm. As this has to be done only once, we keep it out of the main Jupyter Notebook that implements the different methods and models on the training and testing data set.

Three majors steps have to be performed in this Jupyter Notebook:
1. Proper loading of all images from the subdirectories
2. Definition of functions that construct the pipeline
    + Translate `.jpg` picture into pixels with intensity per pixel
    + Creating arrays of corresponding labels
    + Combining all arrays into proper dictionary
    + Store translated pixel data so that main Jupyter Notebook can access the data
3. Running pipeline with different pixel resolutions

<br>
<br>
<br>

## 1. Defintion of where to find files in folders

<br>

In [53]:
# Import necessary libraries and set-up Jupyter Notebook.

# Common imports
import numpy as np
import os
import matplotlib.pyplot as plt
import timeit # for measuring time/code performance
import pickle # for storing dictionary in the end
import PIL #Python Image Library

# to make this notebook's output stable across runs (safety measure)
np.random.seed(42)

In [54]:
# Switch between toy and full data
full_data_switch_on = False #Set True for full data set and False for Dummy Data set (see comment below)

<div class="alert alert-block alert-danger">
<b>Action required</b>
<p>
    
You have to set the switch whether you want to use the full dataset (True) or the dummy toy dataset (False). We set aside 100 correct and 100 incorrect pictures into a dummy toy data set in order to test our code faster. For running the algorithm with the dummy toy data everything is included in the GitHub-Repository (in the folder "01_data/99_dummy_toy_data").
    
However, if you want to run the algorithm with the full data set, you have to download the corresponding files under the Dropbox-Link below. The raw data is placed in the Dropbox folder "00_raw" (Hint: It is ca. 40,5 GB) and you have to download it into the repository folder "01_data/00_raw/".

The reason why we cannot directly use the links here is that we do not have figured out yet how to loop through subfolders and files Dropbox online. GitHub does not allow us to upload such an amount of data.
<br>
Dropbox-Link: https://www.dropbox.com/sh/45vbkq1ihfnhqem/AAADdq6mJKaLsG1w7SDK-QV8a?dl=0    

<br>
<b>
!!!  Be aware: Running this Jupyter Notebook with the full data set requires probably >8 hours of runtime for a 32x32 pixel resolution - depending on your hardware !!!

</b>    
</div>

In [55]:
# Setting path variables depending on switch
if full_data_switch_on == True: 
    # Set path to full data set of correct and incorrect files
    ROOT_DATA = "01_data/00_raw/Masked-Face-Net-Dataset"
    PATH_DATA_CORRECT = os.path.join(ROOT_DATA + "/CMFD")
    PATH_DATA_INCORRECT = os.path.join(ROOT_DATA + "/IMFD")
else:
    # Set path to dummy toy data set of correct and incorrect files
    ROOT_DATA = "01_data/99_dummy_toy_data"
    PATH_DATA_CORRECT = os.path.join(ROOT_DATA + "/correct")
    PATH_DATA_INCORRECT = os.path.join(ROOT_DATA + "/incorrect")

In [56]:
# Defining list of all pictures to include
import os

def list_files(dir):
    r = []
    for root, dirs, files in os.walk(dir):
        for name in files:
            r.append(os.path.join(root, name))
    return r

filenames_correct = list_files(PATH_DATA_CORRECT)
filenames_incorrect = list_files(PATH_DATA_INCORRECT)

In [57]:
fil_cor_array = np.array(filenames_correct)
fil_incor_array = np.array(filenames_incorrect)

pic_ids = np.array([])
pic_ids = np.vstack((fil_cor_array, fil_incor_array))

In [58]:
pic_ids.shape

(2, 100)

In [59]:
filenames_correct[1]

'01_data/99_dummy_toy_data/correct/00010_Mask.jpg'

In [60]:
if PATH_DATA_CORRECT in filenames_correct[1]:
    test = filenames_correct[1].replace(PATH_DATA_CORRECT + "/", "")

test

'00010_Mask.jpg'

In [61]:
pic_ids = np.array([])
for i in range(len(filenames_correct)):
    file_short = filenames_correct[i].replace(PATH_DATA_CORRECT + "/", "")
    pic_ids = np.append(pic_ids, file_short)
for i in range(len(filenames_incorrect)):
    file_short = filenames_incorrect[i].replace(PATH_DATA_INCORRECT + "/", "")
    pic_ids = np.append(pic_ids, file_short)

In [62]:
pic_ids.shape

(200,)

<br>

## 2. Defining necessary functions that construct the pipeline

<br>
<br>
<br>


### Function for transforming .jpg-fils into numerical arrays

<br>

In [63]:
# Importing necessary libraries/functions
from PIL import Image as PIL_Image


# Defining function that performs translation of jpg files into numerical representation
def pixel_transformation(target_pixel, correct_switch):
    # Correct/Incorrect switch
    if correct_switch==True:
            
        # Run-time information
        start_time = timeit.default_timer()
        print(">... Starting pixel transformation for correct pictures for resolution: ", target_pixel, "x", target_pixel)

        # Initialize emtpy array of fitting length
        array_length = target_pixel*target_pixel*3 #Times 3 as we have 3 values (RGB) per pixel
        loaded_pics_correct = np.empty([0,array_length])

        #Running trough all correctly worn image files
        for filename in filenames_correct:
            # open picture
            pic = PIL_Image.open(filename)
            # Reduce size from original format to target format
            pic_resized = pic.resize((target_pixel, target_pixel))
            # Extract RGB data
            pic_data = np.array(pic_resized)
            # Include help array to reshape 3D-array(e.g.: 1024, 1024, 3) into 1D array
            help_array = np.reshape(pic_data,(pic_data.size,))
            # Stack each array onto each other to have one larger array of shape (#obs,#pixels*3)
            loaded_pics_correct = np.vstack((loaded_pics_correct, help_array))
        
        #End run-time information
        elapsed = timeit.default_timer() - start_time
        print("Finished", target_pixel, "x", target_pixel ,"pixel transformation for correct pictures. Run-time in seconds: ", round(elapsed,2))
 
        # Returning
        return loaded_pics_correct
        
  
    elif correct_switch==False:
            
        # Run-time information
        start_time = timeit.default_timer()
        print(">... Starting pixel transformation for incorrect pictures for resolution: ", target_pixel, "x", target_pixel)

        # Initialize emtpy array of fitting length
        array_length = target_pixel*target_pixel*3 #Times 3 as we have 3 values (RGB) per pixel
        loaded_pics_incorrect = np.empty([0,array_length])

        #Running trough all correctly worn image files
        for filename in filenames_incorrect:
            # open picture
            pic = PIL_Image.open(filename)
            # Reduce size from original format to target format
            pic_resized = pic.resize((target_pixel, target_pixel))
            # Extract RGB data
            pic_data = np.array(pic_resized)
            # Include help array to reshape 3D-array(e.g.: 1024, 1024, 3) into 1D array
            help_array = np.reshape(pic_data,(pic_data.size,))
            # Stack each array onto each other to have one larger array of shape (#obs,#pixels*3)
            loaded_pics_incorrect = np.vstack((loaded_pics_incorrect, help_array))

        #End run-time information
        elapsed = timeit.default_timer() - start_time
        print("Finished", target_pixel, "x", target_pixel ,"pixel transformation for incorrect pictures. Run-time in seconds: ", round(elapsed,2))

        # Returning
        return loaded_pics_incorrect
        
   
    else:
        print("Error: Value for correct_switch muste be True or False!")
        
   
    

<br>
<br>
<br>
<br>


### Function for creating label vectors

<br>

In [64]:
# Defining function to create label vector

def add_labels(pic_array, correct_switch):
    # Run-time information
    start_time = timeit.default_timer()
    print(">... Starting to create labels vector for array.")
    
    # Define label depending on switch
    if correct_switch==True:
        label_item = 1
    elif correct_switch==False:
        label_item = 0
    else:
        print("Error: Value for correct_switch muste be True or False!")
        
    # Initialize array and run for-loop
    labels = np.array([])
    for itr in range(len(pic_array)):
        labels = np.append(labels, label_item)
    # Transform to integers
    labels = labels.astype(np.uint8)
    
    # End run-time information
    elapsed = timeit.default_timer() - start_time
    print("Finished to create labels vector for array. Label: ",label_item,". Run-time in seconds: ", round(elapsed,2))
    
    #Returning values
    return labels

<br>
<br>
<br>
<br>


### Function for combining arrays into dictionary

<br>

In [65]:
# Defining function to combine arrays to dictionary

def comb_to_dict(feat_cor, feat_incor, labels_cor, labels_incor):
    # Run-time information
    start_time = timeit.default_timer()
    print(">... Starting to combine arrays to dictionary.")
    
    # combine label arrays
    labels = np.array([])
    labels = np.append(labels_cor, labels_incor)
    
    # Combine feature arrays
    cleaned_data = np.array([])
    cleaned_data = np.vstack((feat_cor, feat_incor))
    
    # Combine pic id arrays
    pic_ids = np.array([])
    for i in range(len(filenames_correct)):
        file_short = filenames_correct[i].replace(PATH_DATA_CORRECT + "/", "")
        pic_ids = np.append(pic_ids, file_short)
    for i in range(len(filenames_incorrect)):
        file_short = filenames_incorrect[i].replace(PATH_DATA_INCORRECT + "/", "")
        pic_ids = np.append(pic_ids, file_short)
    
    # Combine data and labels into one dictionary
    pic_data = {}
    pic_data["rgb_data"] = cleaned_data
    pic_data["labels"] = labels
    pic_data["pic_ids"] = pic_ids
    
    # End run-time information
    elapsed = timeit.default_timer() - start_time
    print("Finished creating dictionary. Run-time in seconds: ", round(elapsed,2))
    
    #Returning values
    return pic_data
    

<br>
<br>
<br>
<br>


### Function for storing dictionary with approriate name

<br>

In [66]:
# Defining function for storing dictionary with appropiate name in appropriate folder
def store_dict(pic_dict, target_pixel):
    if full_data_switch_on == True: 
        file = open(("01_data/01_cleaned/pic_data_full_"+str(target_pixel)+".pkl"),"wb")
        pickle.dump(pic_dict, file)
        file.close()
        print("Successfully stored in: 01_data/01_cleaned/pic_data_full_"+str(target_pixel)+".pkl")
    else: 
        file = open(("01_data/01_cleaned/pic_data_dummy_toy_"+str(target_pixel)+".pkl"),"wb")
        pickle.dump(pic_dict, file)
        file.close()
        print("Successfully stored in: 01_data/01_cleaned/pic_data_dummy_toy_"+str(target_pixel)+".pkl")

<br>
<br>
<br>
<br>


### Function for storing dictionary with approriate name

<br>

In [67]:
# Defining function to run whole pipeline

def pic_pipeline(pixel_res):
    # Run-time information
    start_time_all = timeit.default_timer()
    print("\n\n>... Starting picture pipeline for resolution: ", pixel_res, "x", pixel_res, "\n\n")
    
    # Transformation
    feat_correct = pixel_transformation(target_pixel=pixel_res, correct_switch=True)
    feat_incorrect = pixel_transformation(target_pixel=pixel_res, correct_switch=False)
    # Adding labels
    labels_correct = add_labels(pic_array=feat_correct, correct_switch=True)
    labels_incorrect = add_labels(pic_array=feat_incorrect, correct_switch=False)
    # Combining
    pic_data = comb_to_dict(feat_cor=feat_correct, 
                            feat_incor=feat_incorrect, 
                            labels_cor=labels_correct, 
                            labels_incor=labels_incorrect)
    # Storing
    if store_switch == True:
        store_dict(pic_dict=pic_data, target_pixel=pixel_res)
    elif store_switch == False:
        print("Be aware. New arrays are not stored.")
    else:
        print("Error. Store Switch must be either True or False!")
    
    # End run-time information
    elapsed = timeit.default_timer() - start_time_all
    print("\n\nFinished picture pipeline for resultion: ", pixel_res, "x", pixel_res, "Run-time in seconds:", round(elapsed,2))

<br>
<br>
<br>
<br>

## 3. Running full transformation pipeline

<br>

In [69]:
# Defining pixel resolutions to use: All pics will be reshaped quadratically: So 16 will become 16x16 resolution
desired_pixel_res = [16, 32, 48, 64]

# Define if new run shall be stored (For development of code)
store_switch = True


for pix in desired_pixel_res:
    # Running transformation pipeline incl storing and adding labels
    pic_pipeline(pixel_res=pix)
    print("\n\n Finished overall pipeline!")
    



>... Starting picture pipeline for resolution:  16 x 16 


>... Starting pixel transformation for correct pictures for resolution:  16 x 16
Finished 16 x 16 pixel transformation for correct pictures. Run-time in seconds:  1.6
>... Starting pixel transformation for incorrect pictures for resolution:  16 x 16
Finished 16 x 16 pixel transformation for incorrect pictures. Run-time in seconds:  1.51
>... Starting to create labels vector for array.
Finished to create labels vector for array. Label:  1 . Run-time in seconds:  0.0
>... Starting to create labels vector for array.
Finished to create labels vector for array. Label:  0 . Run-time in seconds:  0.0
>... Starting to combine arrays to dictionary.
Finished creating dictionary. Run-time in seconds:  0.0
Successfully stored in: 01_data/01_cleaned/pic_data_dummy_toy_16.pkl


Finished picture pipeline for resultion:  16 x 16 Run-time in seconds: 3.12


 Finished overall pipeline!


>... Starting picture pipeline for resolution:  32 x 32 